In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os
import numpy as np

# Path ke folder dataset
dataset_path = "./dataset"

# Parameter dasar
image_size = (150, 150)  # Ukuran gambar
batch_size = 32          # Ukuran batch

# Data Augmentation dan Load Data
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalisasi nilai pixel
    validation_split=0.2  # Membagi dataset untuk training dan validasi
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Membuat Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Kompilasi Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Melatih Model
epochs = 10  # Ubah sesuai kebutuhan
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Simpan Model
model.save("document_classifier.h5")


Found 1323 images belonging to 4 classes.
Found 329 images belonging to 4 classes.
Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 18s 359ms/step - accuracy: 0.7446 - loss: 0.7249 - val_accuracy: 0.9392 - val_loss: 0.2167
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 337ms/step - accuracy: 0.9060 - loss: 0.2546 - val_accuracy: 0.9392 - val_loss: 0.1249
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 339ms/step - accuracy: 0.9292 - loss: 0.1996 - val_accuracy: 0.9635 - val_loss: 0.0689
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 340ms/step - accuracy: 0.9413 - loss: 0.1578 - val_accuracy: 0.9878 - val_loss: 0.0446
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 337ms/step - accuracy: 0.9871 - loss: 0.0444 - val_accuracy: 0.9605 - val_loss: 0.1116
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 337ms/step - accuracy: 0.9731 - loss: 0.0979 - val_accuracy: 0.9818 - val_loss: 0.0561
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 337ms/step - accuracy: 0.9920 - loss: 0.0298 - val_accuracy: 0.9818 - val_loss: 0.0552
Epoch 8/10
42/

In [19]:
# Fungsi untuk Klasifikasi Gambar Baru
def classify_image(image_path):
    from tensorflow.keras.preprocessing.image import load_img, img_to_array

    # Load gambar
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi
    predictions = model.predict(img_array)
    class_indices = train_generator.class_indices
    class_names = {v: k for k, v in class_indices.items()}  # Membalik indeks

    # Identifikasi Kelas
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    # Logika untuk klasifikasi "Other" jika confidence rendah
    if confidence < 0.8:  # Threshold dapat disesuaikan
        return "Other", confidence
    else:
        return class_names[predicted_class], confidence

In [ ]:
# Contoh Penggunaan
image_path = "./ktp1.jpg"
predicted_class, confidence = classify_image(image_path)
print(f"Kelas: {predicted_class}, Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Kelas: SIM, Confidence: 1.00
